<a href="https://colab.research.google.com/github/Elwing-Chou/ml0602/blob/main/transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
fn = "/content/drive/MyDrive/additional/train.zip"
zf = zipfile.ZipFile(fn)
zf.extractall()

In [ ]:
import glob
import matplotlib.pyplot as plt
from PIL import Image
fns = glob.glob("train/*")
print(len(fns))
img = Image.open(fns[0])
print(img.size)
plt.imshow(img)

In [11]:
import pandas as pd
dogf =  glob.glob("train/dog.*")
catf =  glob.glob("train/cat.*")
data = pd.DataFrame({
  "path":dogf + catf,
  "target":[0] * len(dogf) + [1] * len(catf)
})
data

,path,target
0,train/dog.10694.jpg,0
1,train/dog.4214.jpg,0
2,train/dog.1140.jpg,0
3,train/dog.7814.jpg,0
4,train/dog.456.jpg,0
...,...,...
24995,train/cat.7656.jpg,1
24996,train/cat.2840.jpg,1
24997,train/cat.9132.jpg,1
24998,train/cat.5773.jpg,1


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
t1 = GlobalAveragePooling2D()(vgg.output)
out = Dense(2, activation="softmax")(t1)
model = Model(inputs=vgg.input, outputs=out)
model.summary()

In [20]:
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
# trainable設定一定要在compile之前
for l in vgg.layers:
    l.trainable = False
layers = [
    GlobalAveragePooling2D(),
    Dense(2, activation="softmax")
]
model = Sequential(vgg.layers + layers)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [21]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer=Adam(),
              metrics=["accuracy"])

In [ ]:
test = np.random.randint(0, 255, size=(3, 32, 32, 3))
# np.array[第一個軸我要哪幾個, 第二個軸我要哪幾個....]
print(test.shape)
print(test[0:2].shape)
print(test[0:2,0:28,0:28,0:2].shape)
print(test[:,:,:,0:2].shape)
print(test[...,0:2].shape)

In [ ]:
import numpy as np
# 圖片如何處理: 不是除255.0, 請always使用你偷來的模型同樣處理方式
from tensorflow.keras.applications.vgg16 import preprocess_input
img = Image.open(fns[0]).convert("RGB").resize((224, 224))
img_np = np.array(img)
preprocess_input(img_np)

In [58]:
# 我們無法使用fit, 1. fit_generator 2.train_on_batch/test_on_batch
# random.randint(0, 2(inclusive)): 0,1,2
# np.random,randint(0, 3(exclusive)): 0,1,2
def get_data(x, y, batch=20):
    idx = np.random.randint(0, len(x), size=batch)
    xidx, yidx = x[idx], y[idx]
    img_ori, img_pre, ans = [], [], []
    for xi, yi in zip(xidx, yidx):
        img = Image.open(xi).convert("RGB").resize((224, 224))
        img_np = np.array(img)
        img_p = preprocess_input(img_np)
        img_ori.append(img_np)
        img_pre.append(img_p)
        ans.append(yi)
    return np.array(img_ori), np.array(img_pre), np.array(ans)

In [59]:
from sklearn.model_selection import train_test_split
x, y = np.array(data["path"]), np.array(data["target"])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
img_ori, img_pre, ans = get_data(x_train, y_train)
print(img_ori.shape)
print(img_pre.shape)
print(ans.shape)

(20, 224, 224, 3)
(20, 224, 224, 3)
(20,)


In [ ]:
for i in range(100):
    print("-" * 15, i, "-" * 15)
    img_ori, img_p, ans = get_data(x_train, y_train)
    result = model.train_on_batch(img_p, ans)
    print("train:", result)
    img_ori, img_p, ans = get_data(x_test, y_test)
    result = model.test_on_batch(img_p, ans)
    print("validate:", result)

In [ ]:
+